In [1]:
import pandas as pd
from datetime import datetime
import plotly.express as px

from firestore_connection import db, DocumentReference, DocumentSnapshot

In [2]:
def load_data() -> pd.DataFrame:
  articles: list[DocumentSnapshot] = db.collection("articles").order_by("created_at").get()
  articles_dict: list[dict[str, str | int | DocumentReference | datetime]] = []
  for article in articles:
    article_data: dict[str, str | datetime | DocumentReference] = article.to_dict()
    user_data: dict[str, str | int] = article_data["author"].get().to_dict()
    del article_data["author"]
    articles_dict.append(article_data | user_data)

  return pd.DataFrame(articles_dict)

In [3]:
df = load_data()
df["created_at"] = pd.to_datetime(df["created_at"]).dt.date
df_filtered = df.copy()

In [7]:
post_per_date: pd.DataFrame = df_filtered.groupby("created_at").size()
post_per_date

created_at
2025-05-28     7
2025-05-29     4
2025-05-30     9
2025-05-31    19
2025-06-01     9
2025-06-02    10
2025-06-03     9
2025-06-04    11
2025-06-05    12
2025-06-06    10
dtype: int64

In [9]:
post_per_date_bar = px.bar(post_per_date, x=post_per_date.index, y=post_per_date.values)
post_per_date_bar

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'created_at=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([datetime.date(2025, 5, 28), datetime.date(2025, 5, 29),
                          datetime.date(2025, 5, 30), datetime.date(2025, 5, 31),
                          datetime.date(2025, 6, 1), datetime.date(2025, 6, 2),
                          datetime.date(2025, 6, 3), datetime.date(2025, 6, 4),
                          datetime.date(2025, 6, 5), datetime.date(2025, 6, 6)], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'BwQJEwkKCQsMCg==', 'dtype': 'i1'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'created_at'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'y'}}}
})

In [15]:
authors = (df.firstname + " " + df.lastname).sort_values().unique()
authors

array(['Ashton Alphandery', 'Francis Lauper', 'Kevin Blackman',
       'Marlène Joris'], dtype=object)